In [ ]:
pip install py-ecg-detectors

In [ ]:
pip install keras.utils.np_utils

ERROR: Could not find a version that satisfies the requirement keras.utils.np_utils (from versions: none)
ERROR: No matching distribution found for keras.utils.np_utils


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from scipy.signal import medfilt
from ecgdetectors import Detectors
from scipy import signal
from scipy.signal import butter, iirnotch, lfilter, filtfilt
import pywt
from pywt import wavedec
import warnings
from skimage.restoration import denoise_wavelet
warnings.filterwarnings('ignore')

ModuleNotFoundError: ignored

In [ ]:
ecg_file = pd.read_csv('ptbdb_normal.csv', header = None)

In [ ]:
c = ecg_file.groupby(187, group_keys = False).apply(lambda ecg_file : ecg_file.sample(1))

In [ ]:
c

In [ ]:
 plt.plot(c.iloc[0,:186])

In [ ]:
def denoise_signal(X, dwt_transform, dlevels, cutoff_low, cutoff_high):
    coeffs = wavedec(X, dwt_transform, level=dlevels)   # wavelet transform 'bior4.4'
    # scale 0 to cutoff_low
    for ca in range(0,cutoff_low):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    # scale cutoff_high to end
    for ca in range(cutoff_high, len(coeffs)):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    Y = pywt.waverec(coeffs, dwt_transform) # inverse wavelet transform
    return Y

In [ ]:
x_noisy = c.iloc[0,:186] + 0.1 * np.random.randn(c.iloc[0,:186].size)
plt.plot(x_noisy)

In [ ]:
x_denoise = denoise_wavelet(x_noisy, method='BayesShrink', mode='soft', wavelet_levels=3, wavelet='sym8', rescale_sigma='True')
plt.plot(x_denoise)

In [ ]:
signal1 = denoise_wavelet(c.iloc[0,:186], method='BayesShrink', mode='soft', wavelet_levels=3, wavelet='sym8', rescale_sigma='True')
plt.plot(signal1)

In [ ]:
def get_median_filter_width(sampling_rate, duration):
    res = int( sampling_rate*duration )
    res += ((res%2) - 1) # needs to be an odd number
    return res
# baseline fitting by filtering
# === Define Filtering Params for Baseline fitting Leads======================
ms_flt_array = [0.02,0.06]    #<-- length of baseline fitting filters (in seconds)
mfa = np.zeros(len(ms_flt_array), dtype='int')
for i in range(0, len(ms_flt_array)):
    mfa[i] = get_median_filter_width(500,ms_flt_array[i])

def filter_signal(X):
    global mfa
    X0 = X  #read orignal signal
    for mi in range(0,len(mfa)):
        X0 = medfilt(X0,mfa[mi]) # apply median filter one by one on top of each other
    X0 = np.subtract(X,X0)  # finally subtract from orignal signal
    return X0

In [ ]:
signal_flt = filter_signal(signal1)
plt.plot(signal_flt)

In [ ]:
# Sampling frequency (replace with your actual value)
fs = 500.0

# Powerline frequency (50 Hz or 60 Hz)
powerline_freq = 50.0  # Change to 60.0 if necessary

# Design a notch filter
f0 = powerline_freq
Q = 30.0
b, a = signal.iirnotch(f0, Q, fs)

# Apply the notch filter
filtered_ecg_signal = signal.lfilter(b, a, signal_flt)

In [ ]:
# Time vector
t = np.arange(0, len(signal_flt)) / fs

# Plot the original and filtered signals
plt.figure(figsize=(12, 6))
plt.plot(t, signal_flt, label="Original ECG Signal")
plt.plot(t, filtered_ecg_signal, label="Filtered ECG Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("ECG Signal with Powerline Interference Removal")
plt.legend()
plt.grid()
plt.show()


In [ ]:
def remove_emg_noise(ecg_signal, fs):
    # Design a band-pass filter to focus on ECG frequencies
    lowcut = 5  # Lower cutoff frequency for ECG
    highcut = 150.0  # Higher cutoff frequency for ECG
    b, a = signal.butter(4, [lowcut, highcut], btype='band', fs=fs)

    # Apply the band-pass filter to remove EMG noise
    filtered_ecg_signal = signal.lfilter(b, a, ecg_signal)

    return filtered_ecg_signal


In [ ]:
plt.plot(signal_flt)

In [ ]:
emg_removed = remove_emg_noise(signal_flt, 500)
plt.plot(emg_removed)

In [ ]:
class Pan_tompkins:
    """ Implementationof Pan Tompkins Algorithm.

    Noise cancellation (bandpass filter) -> Derivative step -> Squaring and integration.

    Params:
        data (array) : ECG data
        sampling rate (int)
    returns:
        Integrated signal (array) : This signal can be used to detect peaks


    ----------------------------------------
    HOW TO USE ?
    Eg.

    ECG_data = [4, 7, 80, 78, 9], sampling  =2000

    call :
       signal = Pan_tompkins(ECG_data, sampling).fit()

    ----------------------------------------

    """
    def __init__(self, data, sample_rate):

        self.data = data
        self.sample_rate = sample_rate


    def fit(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150, window_size=None):
        ''' Fit the signal according to algorithm and returns integrated signal

        '''
        # 1.Noise cancellationusing bandpass filter
        self.filtered_BandPass = self.band_pass_filter(normalized_cut_offs, butter_filter_order, padlen)

        # 2.derivate filter to get slpor of the QRS
        self.derviate_pass = self.derivative_filter()

        # 3.Squaring to enhance dominant peaks in QRS
        self.square_pass = self.squaring()

        # 4.To get info about QRS complex
        self.integrated_signal = self.moving_window_integration( window_size)

        return self.integrated_signal


    def band_pass_filter(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150):
        ''' Band pass filter for Pan tompkins algorithm
            with a bandpass setting of 5 to 20 Hz

            params:
                normalized_cut_offs (list) : bandpass setting canbe changed here
                bandpass filte rorder (int) : deffault 2
                padlen (int) : padding length for data , default = 150
                        scipy default value = 2 * max(len(a coeff, b coeff))

            return:
                filtered_BandPass (array)
        '''

        # Calculate nyquist sample rate and cutoffs
        nyquist_sample_rate = self.sample_rate / 2

        # calculate cutoffs
        if normalized_cut_offs is None:
            normalized_cut_offs = [5/nyquist_sample_rate, 15/nyquist_sample_rate]
        else:
            assert type(self.sample_rate ) is list, "Cutoffs should be a list with [low, high] values"

        # butter coefficinets
        b_coeff, a_coeff = butter(butter_filter_order, normalized_cut_offs, btype='bandpass')[:2]

        # apply forward and backward filter
        filtered_BandPass = filtfilt(b_coeff, a_coeff, self.data, padlen=padlen)

        return filtered_BandPass


    def derivative_filter(self):
        ''' Derivative filter

        params:
            filtered_BandPass (array) : outputof bandpass filter
        return:
            derivative_pass (array)
        '''

        # apply differentiation
        derviate_pass= np.diff(self.band_pass_filter())

        return derviate_pass


    def squaring(self):
        ''' squaring application on derivate filter output data

        params:

        return:
            square_pass (array)
        '''

        # apply squaring
        square_pass= self.derivative_filter() **2

        return square_pass


    def moving_window_integration(self, window_size=None):
        ''' Moving avergae filter

        Params:
            window_size (int) : no. of samples to average, if not provided : 0.08 * sample rate
            sample_rate (int) : should be given if window_size is not given
        return:
            integrated_signal (array)
        '''

        if window_size is None:
            assert self.sample_rate is not None, "if window size is None, sampling rate should be given"
            window_size = int(0.08 * int(self.sample_rate))  # given in paper 150ms as a window size


        # define integrated signal
        integrated_signal = np.zeros_like(self.squaring())

        # cumulative sum of signal
        cumulative_sum = self.squaring().cumsum()

        # estimationof area/ integral below the curve deifnes the data
        integrated_signal[window_size:] = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size

        integrated_signal[:window_size] = cumulative_sum[:window_size] / np.arange(1, window_size + 1)

        return integrated_signal

In [ ]:
signal_pt = Pan_tompkins(signal_flt, 500).fit()

In [ ]:
plt.plot(signal_flt)

In [ ]:
plt.plot(signal_pt)
# plt.plot(signal_flt)
